In [6]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
import csv
import io
import json
import random
import math
warnings.filterwarnings("ignore")

In [7]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
payload = { 'api_key': '13b2927069f6f95d3b84bd69aa7468e2', 'url': 'http://newhouse.591.com.tw' }
request_url='http://newhouse.591.com.tw/home/housing/info?hid=119282'
res=requests.get(request_url, headers = headers,params=payload)
bs=BeautifulSoup(res.text,'html.parser')


In [8]:
# input 建案網址 return 建案建案詳情欄位
# 產出為建案名與16個建案資訊
def getData(url):
    request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers,params=payload)
    #bs=BeautifulSoup(res.text,'html.parser')

    request_url_detail='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()+"/detail"
    res_detail=requests.get(request_url_detail, headers = headers,params=payload)

    if res.status_code == 200 & res_detail.status_code==200:
        bs=BeautifulSoup(res.text,'html.parser')
        bs_detail=BeautifulSoup(res_detail.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        title="NULL"

        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        # 利用 try except 若無撈到資料則寫入NULL
        title=bs.find('h1').text
        try:
          tag = ', '.join([span.text.strip() for span in bs.find("p", {'class':"build-tag"}).find_all("span")]) if bs.find("p", {'class':"build-tag"}) else "未找到相應標籤" #建案標籤
        except:
          tag = "NULL"
        try:
          unit_price=bs.find("span", {'class':"price"}).text #單價
        except:
          unit_price= "NULL"
        try:
          unit = bs.find("span", {'class':"unit"}).text #單位
        except:
          unit = "NULL"
        try:
          material = bs.find("h4",text="建材說明").findNext("p").text #建材說明
        except:
          material= "NULL"
        try:
          htype=bs_detail.find("span", text="建案類別").findNext("p").string.strip() #建案類別
        except:
          htype= "NULL"
        try:
          htype2=bs_detail.find("span", text="建案型態").findNext("p").string.strip().replace(' ', '').replace('\n', '、') #建案型態
        except:
          htype2= "NULL"
        try:
          htype3=bs_detail.find("span", text="公開銷售").findNext("p").string.strip() #公開銷售
        except:
          htype3= "NULL"
        try:
          htype4=bs_detail.find("p", {'class':"address"}).findNext("span").text #基地地址
        except:
          htype4= "NULL"
        try:
          htype5=bs_detail.find("span", text="交屋屋況").findNext("p").string.strip() #交屋屋況
        except:
          htype5= "NULL"
        try:
          htype6=bs_detail.find("span",text="格局規劃").findNext("p").text.strip().replace(' ', '') #格局規劃
        except:
          htype6= "NULL"
        try:
          htype7=bs_detail.find("span", text="投資建設").findNext("p").string.strip() #投資建設
        except:
          htype7= "NULL"
        try:
          htype8=bs_detail.find("span", text="營造公司").findNext("p").string.strip() #營造公司
        except:
          htype8= "NULL"
        try:
          htype9=bs_detail.find("span", text="棟戶規劃").findNext("p").string.strip() #棟戶規劃
        except:
          htype9= "NULL"
        try:
          htype10=bs_detail.find("span", text="樓層規劃").findNext("p").string.strip() #樓層規劃
        except:
          htype10= "NULL"
        try:
          htype11=bs_detail.find("span", text="用途規劃").findNext("p").string.strip() #用途規劃
        except:
          htype11= "NULL"
        return title,tag,unit_price,unit,material,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404

In [9]:
def main(outputfile, rid, sid, totalpages):
    totalpages = totalpages
    print('Total pages: ', totalpages)
    columns = [ "建案名稱", "建案標籤","單價","單位","建材說明" ,"建案類別", "建物形態", "公開銷售", "基地地址", "交屋屋況"\
                        ,"格局規劃", "投資建設", "營造公司", "棟戶規劃", "樓層規劃", "用途規劃", "網址"]
    df = pd.DataFrame(columns=columns)
    for i in range(1, totalpages+1):
                request_url = "https://newhouse.591.com.tw/home/housing/search?rid="+str(rid)+"&sid="+str(sid)+"&page="+str(i)
                response = requests.get(request_url, headers = headers,params=payload)
                response = response.json()
                items = response["data"]["items"]

                house_url_list=[] #存放網址list
                for key in items:
                    id = key["hid"] # 每個物件的 id
                    house_url_list.append(id)
                time.sleep(random.randint(5,15)) #反爬蟲

                for url in house_url_list:
                    title,tag,unit_price,unit,material,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11 = getData(url)
                    # 創建新的DataFrame
                    new_row = pd.DataFrame([[title, tag, unit_price, unit, material, htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11, str('https://newhouse.591.com.tw/home/housing/info?hid='+str(url))]],
                                            columns=columns)
                    # 將新的DataFrame添加到原有的df中
                    df = df.append(new_row, ignore_index=True)
                # ------------------------------------------ #
                print(i/totalpages*100, '%') # print out 完成 %數
    df.to_excel(outputfile)

In [10]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    # 以台北市不限區舉例(預設網址可能沒寫rid&sid, 點選縣市或區往只會顯示如下)
    # link:https://newhouse.591.com.tw/housing-list.html?rid=1&sid=0

    request_url = "https://newhouse.591.com.tw/home/housing/search?rid=0&sid=0"
    headers={
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
        }
    response = requests.get(request_url, headers = headers)
    response = response.json()
    total = response["data"]["total"]
    per_page = response["data"]["per_page"]

    output_file_name = 'E:\CodeProject\591WebCrawler\newhouse591_data.excel' #設定存放位置與檔名
    rid = 0           # 設定縣市 (台北市 rid = 1)
    sid = 0           # 設定地區 (不限區 sid = 0)
    totalpages = math.ceil(total/per_page)    # 設定抓取頁數
    # ---------------------------------------- #

    main(output_file_name, rid, sid, totalpages)                                          #匯出excel檔
    print('\nfinish!')

Total pages:  782
0.1278772378516624 %
0.2557544757033248 %
0.3836317135549872 %
0.5115089514066496 %
0.639386189258312 %
0.7672634271099744 %
0.8951406649616368 %
1.0230179028132993 %
1.1508951406649617 %
1.278772378516624 %
1.4066496163682864 %
1.5345268542199488 %
1.6624040920716114 %
1.7902813299232736 %
1.9181585677749362 %
2.0460358056265986 %
2.1739130434782608 %
2.3017902813299234 %
2.4296675191815855 %
2.557544757033248 %
2.6854219948849107 %
2.813299232736573 %
2.941176470588235 %
3.0690537084398977 %
3.1969309462915603 %
3.324808184143223 %
3.4526854219948846 %
3.580562659846547 %
3.70843989769821 %
3.8363171355498724 %
3.9641943734015346 %
4.092071611253197 %
4.219948849104859 %
4.3478260869565215 %
4.475703324808184 %
4.603580562659847 %
4.731457800511509 %
4.859335038363171 %
4.987212276214834 %
5.115089514066496 %
5.242966751918159 %
5.3708439897698215 %
5.498721227621484 %
5.626598465473146 %
5.754475703324808 %
5.88235294117647 %
6.010230179028133 %
6.138107416879795 %

ConnectionError: HTTPSConnectionPool(host='newhouse.591.com.tw', port=443): Max retries exceeded with url: /home/housing/info?hid=130150/detail&api_key=13b2927069f6f95d3b84bd69aa7468e2&url=http%3A%2F%2Fnewhouse.591.com.tw (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BF491CECA0>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))